In [1]:
import pandas as pd
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb

Loading configurations from /workspaces/DuckDB-fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


Powerball is a popular lottery game in the United States. Players choose five numbers from 1 to 69 and a Powerball number from 1 to 26. A player wins the jackpot by matching all five numbers plus the Powerball number. Learn more about Powerball [here](https://en.wikipedia.org/wiki/Powerball). 

In this bonus exercise, we'll look at New York lottery powerball data! We'll pull it into a dataframe by reading directly from ny.gov's website.

In [2]:
powerball_df = pd.read_csv("https://data.ny.gov/api/views/d6yy-54nr/rows.csv")

powerball_df.rename(
    columns={k: k.lower().replace(" ", "_") for k in powerball_df.columns}, inplace=True
)

The winning numbers are made up of five "white balls" from a matrix of 69 and one "Powerball" from a matrix of 26, resulting in jackpot odds of 1 in 292,201,338 per play. Let's take a look at the data:

In [3]:
%%sql
SELECT * FROM powerball_df LIMIT 5

,draw_date,winning_numbers,multiplier
0,09/26/2020,11 21 27 36 62 24,3.0
1,09/30/2020,14 18 36 49 67 18,2.0
2,10/03/2020,18 31 36 43 47 20,2.0
3,10/07/2020,06 24 30 53 56 19,2.0
4,10/10/2020,05 18 23 40 50 18,3.0


Write a query that splits the winning numbers into separate columns. Your query should return a result with columns: `draw_date`, `num1`, `num2`, `num3`, `num4`, `num5`, `num6`, and `multiplier`

In [4]:
%%sql
SELECT
    draw_date,
    SPLIT_PART(winning_numbers, ' ', 1) AS num1,
    SPLIT_PART(winning_numbers, ' ', 2) AS num2,
    SPLIT_PART(winning_numbers, ' ', 3) AS num3,
    SPLIT_PART(winning_numbers, ' ', 4) AS num4,
    SPLIT_PART(winning_numbers, ' ', 5) AS num5,
    SPLIT_PART(winning_numbers, ' ', 6) AS num6,
    multiplier::INT
FROM powerball_df
LIMIT 5

,draw_date,num1,num2,num3,num4,num5,num6,CAST(multiplier AS INTEGER)
0,09/26/2020,11,21,27,36,62,24,3
1,09/30/2020,14,18,36,49,67,18,2
2,10/03/2020,18,31,36,43,47,20,2
3,10/07/2020,06,24,30,53,56,19,2
4,10/10/2020,05,18,23,40,50,18,3


Using the above as a base, write a new query that returns a table where each drawn number represents a row and each column is the count of occurrences where that number was drawn in the proper position. Your response should look like this:

| range_str | num1_ct | num2_ct | num3_ct | num4_ct | num5_ct | num6_ct |
|----------:|--------:|--------:|--------:|--------:|--------:|--------:|
|        01 |     121 |       0 |       0 |       0 |       0 |      54 |
|        02 |     112 |       9 |       0 |       0 |       0 |      51 |
|        03 |     106 |      18 |       1 |       0 |       0 |      52 |
|        04 |      90 |      22 |       0 |       0 |       0 |      64 |
|        05 |      96 |      17 |       0 |       0 |       0 |      59 |

Hints:
- The numbers aren't actually numbers— they're left padded strings. 
- We can't be sure every number has been drawn to create the "index" (range_str)— it might be best to generate the index instead.

In [5]:
%%sql
WITH range AS (
    SELECT
        r.range,
        lpad(r.range::TEXT, 2, '0') AS range_str
    FROM range(1,70) r
    ), nums AS (
    SELECT
        draw_date,
        SPLIT_PART(winning_numbers, ' ', 1) AS num1,
        SPLIT_PART(winning_numbers, ' ', 2) AS num2,
        SPLIT_PART(winning_numbers, ' ', 3) AS num3,
        SPLIT_PART(winning_numbers, ' ', 4) AS num4,
        SPLIT_PART(winning_numbers, ' ', 5) AS num5,
        SPLIT_PART(winning_numbers, ' ', 6) AS num6,
        multiplier::INT as multiplier
    FROM powerball_df
)
SELECT
    range.range_str,

    COUNT(DISTINCT CASE WHEN range.range_str = num1 THEN draw_date END) AS num1_ct,
    COUNT(DISTINCT CASE WHEN range.range_str = num2 THEN draw_date END) AS num2_ct,
    COUNT(DISTINCT CASE WHEN range.range_str = num3 THEN draw_date END) AS num3_ct,
    COUNT(DISTINCT CASE WHEN range.range_str = num4 THEN draw_date END) AS num4_ct,
    COUNT(DISTINCT CASE WHEN range.range_str = num5 THEN draw_date END) AS num5_ct,
    COUNT(DISTINCT CASE WHEN range.range_str = num6 THEN draw_date END) AS num6_ct,
FROM nums
CROSS JOIN range
GROUP BY 1
ORDER BY 1 ASC

,range_str,num1_ct,num2_ct,num3_ct,num4_ct,num5_ct,num6_ct
0,01,139,0,0,0,0,66
1,02,125,12,0,0,0,61
2,03,120,19,2,0,0,61
3,04,110,24,0,0,0,73
4,05,112,18,0,0,0,72
...,...,...,...,...,...,...,...
64,65,0,0,1,15,62,0
65,66,0,0,0,24,64,0
66,67,0,0,0,11,82,0
67,68,0,0,0,7,82,0


Modify the previous query to return the _most_ common number for each draw. Your result should look something like this:

| most_popular_num1 | most_popular_num2 | most_popular_num3 | most_popular_num4 | most_popular_num5 | most_popular_num6 |
|-------------------|-------------------|-------------------|-------------------|-------------------|-------------------|
| num_1             | num_2             | num_3             | num_4             | num_5             | num_6             |

In [6]:
%%sql
WITH range AS (
    SELECT
        r.range,
        lpad(r.range::TEXT, 2, '0') AS range_str
    FROM range(1,70) r
    ), nums AS (
    SELECT
        draw_date,
        SPLIT_PART(winning_numbers, ' ', 1) AS num1,
        SPLIT_PART(winning_numbers, ' ', 2) AS num2,
        SPLIT_PART(winning_numbers, ' ', 3) AS num3,
        SPLIT_PART(winning_numbers, ' ', 4) AS num4,
        SPLIT_PART(winning_numbers, ' ', 5) AS num5,
        SPLIT_PART(winning_numbers, ' ', 6) AS num6,
        multiplier::INT as multiplier
    FROM powerball_df
), num_counts AS (
    SELECT
        range.range_str,

        COUNT(DISTINCT CASE WHEN range.range_str = num1 THEN draw_date END) AS num1_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num2 THEN draw_date END) AS num2_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num3 THEN draw_date END) AS num3_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num4 THEN draw_date END) AS num4_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num5 THEN draw_date END) AS num5_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num6 THEN draw_date END) AS num6_ct,
    FROM nums
    CROSS JOIN range
    GROUP BY 1
    ORDER BY 2 DESC
)
SELECT
    DISTINCT
    FIRST_VALUE(range_str) OVER (ORDER BY num1_ct DESC) AS most_popular_num1,
    FIRST_VALUE(range_str) OVER (ORDER BY num2_ct DESC) AS most_popular_num2,
    FIRST_VALUE(range_str) OVER (ORDER BY num3_ct DESC) AS most_popular_num3,
    FIRST_VALUE(range_str) OVER (ORDER BY num4_ct DESC) AS most_popular_num4,
    FIRST_VALUE(range_str) OVER (ORDER BY num5_ct DESC) AS most_popular_num5,
    FIRST_VALUE(range_str) OVER (ORDER BY num6_ct DESC) AS most_popular_num6,
FROM num_counts

,most_popular_num1,most_popular_num2,most_popular_num3,most_popular_num4,most_popular_num5,most_popular_num6
0,01,12,37,52,69,24


In addition to returning the most popular number, return the percentage of time that number was drawn

In [7]:
%%sql
WITH range AS (
    SELECT
        r.range,
        lpad(r.range::TEXT, 2, '0') AS range_str
    FROM range(1,70) r
    ), nums AS (
    SELECT
        draw_date,
        SPLIT_PART(winning_numbers, ' ', 1) AS num1,
        SPLIT_PART(winning_numbers, ' ', 2) AS num2,
        SPLIT_PART(winning_numbers, ' ', 3) AS num3,
        SPLIT_PART(winning_numbers, ' ', 4) AS num4,
        SPLIT_PART(winning_numbers, ' ', 5) AS num5,
        SPLIT_PART(winning_numbers, ' ', 6) AS num6,
        multiplier::INT as multiplier
    FROM powerball_df
), num_draws AS (
    SELECT COUNT(*) as num_draws FROM powerball_df
), num_counts AS (
    SELECT
        range.range_str,
        num_draws.num_draws,
        COUNT(DISTINCT CASE WHEN range.range_str = num1 THEN draw_date END) AS num1_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num2 THEN draw_date END) AS num2_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num3 THEN draw_date END) AS num3_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num4 THEN draw_date END) AS num4_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num5 THEN draw_date END) AS num5_ct,
        COUNT(DISTINCT CASE WHEN range.range_str = num6 THEN draw_date END) AS num6_ct,

        COUNT(DISTINCT CASE WHEN range.range_str = num1 THEN draw_date END) / num_draws AS num1_pct,
        COUNT(DISTINCT CASE WHEN range.range_str = num2 THEN draw_date END) / num_draws AS num2_pct,
        COUNT(DISTINCT CASE WHEN range.range_str = num3 THEN draw_date END) / num_draws AS num3_pct,
        COUNT(DISTINCT CASE WHEN range.range_str = num4 THEN draw_date END) / num_draws AS num4_pct,
        COUNT(DISTINCT CASE WHEN range.range_str = num5 THEN draw_date END) / num_draws AS num5_pct,
        COUNT(DISTINCT CASE WHEN range.range_str = num6 THEN draw_date END) / num_draws AS num6_pct,
    FROM nums
    CROSS JOIN range
    CROSS JOIN num_draws
    GROUP BY 1,2
    ORDER BY 2 DESC
), agg AS (
SELECT
    DISTINCT
        FIRST_VALUE(range_str) OVER (ORDER BY num1_ct DESC) AS most_popular_num1,
        FIRST_VALUE(range_str) OVER (ORDER BY num2_ct DESC) AS most_popular_num2,
        FIRST_VALUE(range_str) OVER (ORDER BY num3_ct DESC) AS most_popular_num3,
        FIRST_VALUE(range_str) OVER (ORDER BY num4_ct DESC) AS most_popular_num4,
        FIRST_VALUE(range_str) OVER (ORDER BY num5_ct DESC) AS most_popular_num5,
        FIRST_VALUE(range_str) OVER (ORDER BY num6_ct DESC) AS most_popular_num6,
        
        CASE FIRST_VALUE(range_str) OVER (ORDER BY num1_ct DESC) WHEN range_str THEN num1_pct END  AS most_popular_num1_pct,
        CASE FIRST_VALUE(range_str) OVER (ORDER BY num2_ct DESC) WHEN range_str THEN num2_pct END  AS most_popular_num2_pct,
        CASE FIRST_VALUE(range_str) OVER (ORDER BY num3_ct DESC) WHEN range_str THEN num3_pct END  AS most_popular_num3_pct,
        CASE FIRST_VALUE(range_str) OVER (ORDER BY num4_ct DESC) WHEN range_str THEN num4_pct END  AS most_popular_num4_pct,
        CASE FIRST_VALUE(range_str) OVER (ORDER BY num5_ct DESC) WHEN range_str THEN num5_pct END  AS most_popular_num5_pct,
        CASE FIRST_VALUE(range_str) OVER (ORDER BY num6_ct DESC) WHEN range_str THEN num6_pct END  AS most_popular_num6_pct
FROM num_counts
)
SELECT
    DISTINCT
    most_popular_num1,
    most_popular_num2,
    most_popular_num3,
    most_popular_num4,
    most_popular_num5,
    most_popular_num6,
    
    MAX(most_popular_num1_pct) AS num1_pct,
    MAX(most_popular_num2_pct) AS num2_pct,
    MAX(most_popular_num3_pct) AS num3_pct,
    MAX(most_popular_num4_pct) AS num4_pct,
    MAX(most_popular_num5_pct) AS num5_pct,
    MAX(most_popular_num6_pct) AS num6_pct,

    MAX(most_popular_num1_pct) *
    MAX(most_popular_num2_pct) *
    MAX(most_popular_num3_pct) *
    MAX(most_popular_num4_pct) *
    MAX(most_popular_num5_pct) *
    MAX(most_popular_num6_pct) as pct_product
FROM agg
GROUP BY 1,2,3,4,5,6

,most_popular_num1,most_popular_num2,most_popular_num3,most_popular_num4,most_popular_num5,most_popular_num6,num1_pct,num2_pct,num3_pct,num4_pct,num5_pct,num6_pct,pct_product
0,01,12,37,52,69,24,0.075667,0.039194,0.034839,0.037561,0.058792,0.042461,9.688126e-09


Have the most popular numbers ever been drawn sequentially? (don't overthink this one)

In [8]:
%%sql
SELECT
    *
FROM powerball_df 
WHERE winning_numbers = '01 12 37 45 59 24'
# :(

,draw_date,winning_numbers,multiplier


Now write a query that, for each draw number, returns the first date. The output should look something like:

| range_str |      num_1 |      num_2 |      num_3 | num_4 | num_5 |      num_6 |
|----------:|-----------:|-----------:|-----------:|------:|------:|-----------:|
|        01 | 2010-04-24 |        NaT |        NaT |   NaT |   NaT | 2010-02-13 |
|        02 | 2010-05-19 | 2014-01-22 |        NaT |   NaT |   NaT | 2010-07-28 |
|        03 | 2010-07-03 | 2010-05-29 | 2019-02-09 |   NaT |   NaT | 2010-07-03 |
|        04 | 2010-02-24 | 2011-03-12 |        NaT |   NaT |   NaT | 2010-02-06 |
|        05 | 2010-02-10 | 2011-01-26 |        NaT |   NaT |   NaT | 2010-04-24 |
|        06 | 2010-03-13 | 2010-05-26 | 2015-03-28 |   NaT |   NaT | 2010-06-16 |

In [9]:
%%sql
WITH range AS (
    SELECT
        r.range,
        lpad(r.range::TEXT, 2, '0') AS range_str
    FROM range(1,70) r
    ), nums AS (
    SELECT
        strptime(draw_date, '%m/%d/%Y')::DATE AS draw_date,
        SPLIT_PART(winning_numbers, ' ', 1) AS num1,
        SPLIT_PART(winning_numbers, ' ', 2) AS num2,
        SPLIT_PART(winning_numbers, ' ', 3) AS num3,
        SPLIT_PART(winning_numbers, ' ', 4) AS num4,
        SPLIT_PART(winning_numbers, ' ', 5) AS num5,
        SPLIT_PART(winning_numbers, ' ', 6) AS num6,
        multiplier::INT as multiplier
    FROM powerball_df
), n1 AS (
SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num1) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num1
), n2 AS (
SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num2) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num2
), n3 AS (
SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num3) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num3
), n4 AS (
    SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num4) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num4
), n5 AS (
    SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num5) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num5
), n6 AS (
    SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num6) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num6
)
SELECT
    range.range_str,
    n1.num as num_1,
    n2.num as num_2,
    n3.num as num_3,
    n4.num as num_4,
    n5.num as num_5,
    n6.num as num_6

FROM range
INNER JOIN n1
    ON n1.range_str = range.range_str
INNER JOIN n2
    ON n2.range_str = range.range_str
INNER JOIN n3
    ON n3.range_str = range.range_str
INNER JOIN n4
    ON n4.range_str = range.range_str
INNER JOIN n5
    ON n5.range_str = range.range_str
INNER JOIN n6
    ON n6.range_str = range.range_str
ORDER BY range.range_str
LIMIT 50

,range_str,num_1,num_2,num_3,num_4,num_5,num_6
0,01,2010-04-24,NaT,NaT,NaT,NaT,2010-02-13
1,02,2010-05-19,2014-01-22,NaT,NaT,NaT,2010-07-28
2,03,2010-07-03,2010-05-29,2019-02-09,NaT,NaT,2010-07-03
3,04,2010-02-24,2011-03-12,NaT,NaT,NaT,2010-02-06
4,05,2010-02-10,2011-01-26,NaT,NaT,NaT,2010-04-24
5,06,2010-03-13,2010-05-26,2015-03-28,NaT,NaT,2010-06-16
6,07,2010-02-17,2010-05-19,2012-08-25,2019-02-09,NaT,2010-03-24
7,08,2010-06-16,2010-02-10,2012-09-19,NaT,NaT,2010-03-13
8,09,2010-03-20,2010-03-03,2012-11-03,2014-01-22,NaT,2010-03-20
9,10,2010-02-13,2010-07-03,2010-05-26,NaT,NaT,2010-03-27


Can you find the first drawn set of numbers by windowing over the previous result?

In [10]:
%%sql
WITH range AS (
    SELECT
        r.range,
        lpad(r.range::TEXT, 2, '0') AS range_str
    FROM range(1,70) r
    ), nums AS (
    SELECT
        strptime(draw_date, '%m/%d/%Y')::DATE AS draw_date,
        SPLIT_PART(winning_numbers, ' ', 1) AS num1,
        SPLIT_PART(winning_numbers, ' ', 2) AS num2,
        SPLIT_PART(winning_numbers, ' ', 3) AS num3,
        SPLIT_PART(winning_numbers, ' ', 4) AS num4,
        SPLIT_PART(winning_numbers, ' ', 5) AS num5,
        SPLIT_PART(winning_numbers, ' ', 6) AS num6,
        multiplier::INT as multiplier
    FROM powerball_df
), n1 AS (
SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num1) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num1
), n2 AS (
SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num2) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num2
), n3 AS (
SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num3) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num3
), n4 AS (
    SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num4) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num4
), n5 AS (
    SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num5) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num5
), n6 AS (
    SELECT
    DISTINCT 
    range.range_str,
    MIN(nums.draw_date) OVER (PARTITION BY nums.num6) AS num,
FROM range
LEFT JOIN nums
    ON range.range_str = nums.num6
), agg AS (
    SELECT
        range.range_str,
        n1.num as num_1,
        n2.num as num_2,
        n3.num as num_3,
        n4.num as num_4,
        n5.num as num_5,
        n6.num as num_6

    FROM range
    INNER JOIN n1
        ON n1.range_str = range.range_str
    INNER JOIN n2
        ON n2.range_str = range.range_str
    INNER JOIN n3
        ON n3.range_str = range.range_str
    INNER JOIN n4
        ON n4.range_str = range.range_str
    INNER JOIN n5
        ON n5.range_str = range.range_str
    INNER JOIN n6
        ON n6.range_str = range.range_str
)
SELECT 
    DISTINCT
    FIRST_VALUE(range_str) OVER (ORDER BY num_1 ASC) AS num_1_min,
    FIRST_VALUE(range_str) OVER (ORDER BY num_2 ASC) AS num_2_min,
    FIRST_VALUE(range_str) OVER (ORDER BY num_3 ASC) AS num_3_min,
    FIRST_VALUE(range_str) OVER (ORDER BY num_4 ASC) AS num_4_min,
    FIRST_VALUE(range_str) OVER (ORDER BY num_5 ASC) AS num_5_min,
    FIRST_VALUE(range_str) OVER (ORDER BY num_6 ASC) AS num_6_min
FROM agg

,num_1_min,num_2_min,num_3_min,num_4_min,num_5_min,num_6_min
0,17,22,36,37,52,24


Are those the right numbers?

In [11]:
%%sql
WITH min_date AS (
    SELECT 
        MIN(strptime(draw_date, '%m/%d/%Y')::DATE) as min_date
    FROM powerball_df
)
SELECT 
    * 
FROM powerball_df 
INNER JOIN min_date 
    ON strptime(draw_date, '%m/%d/%Y')::DATE = min_date.min_date

,draw_date,winning_numbers,multiplier,min_date
0,02/03/2010,17 22 36 37 52 24,2.0,2010-02-03
